At this point I have already:
- created a 3:2 sized image in MJ
- downloaded that image to 'downloads' folder locally, save as 'imagename_1x.png'

What this notebook will do:

MJ Image
- take image, rescale to 4:3 & 5:4 aspect ratios; and copy 1x image
- ensure two rescaled images and copied 1x image all have naming scheme 'imagename_AR.png' , be sure to replace AR with whatever aspect ratio it is being converted or copied to. Thus, the 1x image being copied will always be name 'imagename_32.png'
- UPSCAYL APP : run the download folder with 4 images through the pixel upscayl app
- run code to take images out of the folder within the path: /Users/Matt/Pictures/Midjourney/Imagine/Downloads , and replace them with the images already at this path
- run the DPI code on this path /Users/Matt/Pictures/Midjourney/Imagine/Downloads
- 4 images remain in the download folder

Mockup App Process

- 10 or so images should now be in the download folder after the mockups have been created
- all images should be named correctly
- create a folder within the prelisting folder based on the first name of the image 'imagename' in 'imagename_1x.png' from above
- move these images to that folder 
- within that 'imagename' folder, create two folders 'etsy' and 'google docs'
- sort the images with 'mockups' in their saved name and images with '1x' in their name to the etsy folder, the rest to the google docs folder

- Done

- download image from midjourney
- run first cell of code
- upscayl download folder
- run mockup app
- run second cell of code

In [8]:
from PIL import Image
import os
import shutil

def analyze_image_name(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith("_1x.png"):
                return file, os.path.join(root, file)
    return None, None

def resize_image(image_path, aspect_ratio, new_name):
    with Image.open(image_path) as img:
        width, height = img.size
        
        # Calculate new dimensions
        if aspect_ratio == (4, 3):
            new_height = int(width * 3 / 4)
            resized_img = img.resize((width, new_height), Image.LANCZOS)
        elif aspect_ratio == (5, 4):
            new_height = int(width * 4 / 5)
            resized_img = img.resize((width, new_height), Image.LANCZOS)
        else:
            raise ValueError("Unsupported aspect ratio")
        
        # Save the resized image
        resized_img.save(new_name)
        print(f"Saved resized image: {new_name}")

def process_images(directory):
    image_name, file_path = analyze_image_name(directory)
    
    if not image_name or not file_path:
        print("No _1x.png image found.")
        return
    
    base_name = image_name.split('_1x')[0]
    
    # Create new names for the resized and copied images
    new_name_43 = os.path.join(directory, f"{base_name}_43.png")
    new_name_54 = os.path.join(directory, f"{base_name}_54.png")
    copied_name_32 = os.path.join(directory, f"{base_name}_32.png")
    
    # Resize and save the images
    resize_image(file_path, (4, 3), new_name_43)
    resize_image(file_path, (5, 4), new_name_54)
    
    # Copy the original image to a new file with the _32 naming scheme
    shutil.copy(file_path, copied_name_32)
    print(f"Copied original image to: {copied_name_32}")

# Path to your download directory
download_directory = '/Users/matt/Pictures/Midjourney/Imagine/Downloads'

# Process images for resizing and copying
process_images(download_directory)


Saved resized image: /Users/matt/Pictures/Midjourney/Imagine/Downloads/hayride_43.png
Saved resized image: /Users/matt/Pictures/Midjourney/Imagine/Downloads/hayride_54.png
Copied original image to: /Users/matt/Pictures/Midjourney/Imagine/Downloads/hayride_32.png


In [9]:
import os
import shutil
from PIL import Image

def move_and_replace_images_in_directory(directory):
    # Find the new folder within the download directory
    subfolders = [f.path for f in os.scandir(directory) if f.is_dir()]
    if not subfolders:
        print("No subfolder found in the downloads directory.")
        return

    new_folder = subfolders[0]  # Assuming there's only one subfolder
    print(f"Processing folder: {new_folder}")

    # Move images back to the parent directory and replace if names match
    for file_name in os.listdir(new_folder):
        source_path = os.path.join(new_folder, file_name)
        destination_path = os.path.join(directory, file_name)
        
        if os.path.exists(destination_path):
            os.remove(destination_path)
            print(f"Replaced: {destination_path}")
        
        shutil.move(source_path, directory)
        print(f"Moved: {source_path} to {directory}")
    
    # Safely delete the new folder, ensuring it's not named 'downloads'
    if os.path.basename(new_folder).lower() != 'downloads':
        shutil.rmtree(new_folder)
        print(f"Deleted folder: {new_folder}")
    else:
        print("Safety check: Attempted to delete a folder named 'downloads'. Aborting deletion.")

def set_image_dpi(image_path, dpi=(301, 301)):
    try:
        with Image.open(image_path) as img:
            img.save(image_path, dpi=dpi)
            print(f'DPI set to {dpi[0]}x{dpi[1]} for {image_path}')
    except Exception as e:
        print(f"An error occurred: {e}")

def adjust_dpi_for_selected_images(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(suffix in file for suffix in ["_43.png", "_54.png", "_32.png"]):
                file_path = os.path.join(root, file)
                set_image_dpi(file_path)

def organize_images(directory, prelisting_folder):
    image_names = [file for file in os.listdir(directory) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]
    
    # Determine base name from the first image with "_1x.png"
    base_name = None
    for image_name in image_names:
        if "_1x.png" in image_name:
            base_name = image_name.split('_1x')[0]
            break

    if not base_name:
        print("No _1x.png image found.")
        return
    
    # Create directories for organizing images
    new_dir = os.path.join(prelisting_folder, base_name)
    etsy_dir = os.path.join(new_dir, 'etsy')
    google_docs_dir = os.path.join(new_dir, 'google_docs')
    os.makedirs(etsy_dir, exist_ok=True)
    os.makedirs(google_docs_dir, exist_ok=True)
    
    for image_name in image_names:
        file_path = os.path.join(directory, image_name)
        
        if "_1x.png" in image_name or "mockup" in image_name.lower():
            shutil.move(file_path, os.path.join(etsy_dir, image_name))
            print(f"Moved {image_name} to {etsy_dir}")
        elif any(suffix in image_name for suffix in ["_43.png", "_54.png", "_32.png"]):
            shutil.move(file_path, os.path.join(google_docs_dir, image_name))
            print(f"Moved {image_name} to {google_docs_dir}")

# Paths
download_directory = '/Users/matt/Pictures/Midjourney/Imagine/Downloads'
prelisting_folder = '/Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Impasto/Prelisting'

# Move and replace images, then delete the subfolder
move_and_replace_images_in_directory(download_directory)

# Adjust DPI for selected images
adjust_dpi_for_selected_images(download_directory)

# Organize images into folders
organize_images(download_directory, prelisting_folder)


Processing folder: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_8x
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/hayride_1x.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_8x/hayride_1x.png to /Users/matt/Pictures/Midjourney/Imagine/Downloads
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/hayride_32.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_8x/hayride_32.png to /Users/matt/Pictures/Midjourney/Imagine/Downloads
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/hayride_54.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_8x/hayride_54.png to /Users/matt/Pictures/Midjourney/Imagine/Downloads
Replaced: /Users/matt/Pictures/Midjourney/Imagine/Downloads/hayride_43.png
Moved: /Users/matt/Pictures/Midjourney/Imagine/Downloads/upscayl_png_realesrgan-x4plus_8x/hayride_43.png to /Users/matt/Pict

/Users/matt/Documents/Coding/Midjourney/venv/lib/python3.12/site-packages/PIL/Image.py:3368: DecompressionBombWarning: Image size (92467200 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


DPI set to 301x301 for /Users/matt/Pictures/Midjourney/Imagine/Downloads/hayride_54.png
DPI set to 301x301 for /Users/matt/Pictures/Midjourney/Imagine/Downloads/hayride_43.png
Moved hayride_bathroom_mockup.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Impasto/Prelisting/hayride/etsy
Moved hayride_hallway_mockup.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Impasto/Prelisting/hayride/etsy
Moved hayride_1x.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Impasto/Prelisting/hayride/etsy
Moved hayride_office_mockup.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Impasto/Prelisting/hayride/etsy
Moved hayride_bedroom_mockup.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Impasto/Prelisting/hayride/etsy
Moved hayride_kitchen_mockup.png to /Users/matt/Pictures/Midjourney/Digital Products/DoubleVision/Impasto/Prelisting/hayride/etsy
Moved hayride_32.png to /Users/matt/Pictures/Midjourney/